**ANALYSIS NOTEBOOK**

In [2]:
#Evaluation and visualization for analysis of performance
import os
import sys
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import torch

In [14]:
#Plotting the logs of the history files(compatible with .json and .pkl)
def plot_training_history(history_path, output_path):
    #Determine file extension
    _, ext = os.path.splitext(history_path)
    
    #Load history based on file type
    if ext == '.json':
        import json
        with open(history_path, 'r') as f:
            history = json.load(f)
    elif ext == '.pkl':
        import pickle
        with open(history_path, 'rb') as f:
            history = pickle.load(f)
    else:
        raise ValueError("Unsupported file format. Use .json or .pkl")
    
    #Convert to DataFrame for easier plotting
    if isinstance(history, dict):
        # If history is a dict with lists as values, convert directly
        df = pd.DataFrame(history)
    else:
        # If history is a list of dicts, convert to DataFrame
        df = pd.DataFrame(history)
    
    # Check for required columns (handle different naming conventions)
    train_col = next((col for col in df.columns if 'train' in col.lower() and 'acc' in col.lower()), None)
    val_col = next((col for col in df.columns if 'val' in col.lower() and 'acc' in col.lower()), None)
    
    if train_col is None or val_col is None:
        print(f"Warning: Could not find train/val accuracy columns. Available: {list(df.columns)}")
        return
    
    #Plot training and validation accuracy
    plt.figure(figsize=(10, 5))
    if 'epoch' in df.columns:
        plt.plot(df['epoch'], df[train_col], label='Train Accuracy', marker='o')
        plt.plot(df['epoch'], df[val_col], label='Validation Accuracy', marker='s')
    else:
        plt.plot(df[train_col], label='Train Accuracy', marker='o')
        plt.plot(df[val_col], label='Validation Accuracy', marker='s')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.title(f'Training and Validation Accuracy - {os.path.basename(history_path)}')
    plt.legend()
    plt.grid(True)
    plt.savefig(output_path, dpi=100, bbox_inches='tight')
    plt.close()

In [17]:
distilled_model_history_path = 'C:\\Users\\enapa\\Desktop\\Academic City\\Level 400\\DeepExam\\projects-Deep-Learning\\lightvision\\outputs\\logs\\distillation_history.json'
pruned_model_history_path = 'C:\\Users\\enapa\\Desktop\\Academic City\\Level 400\\DeepExam\\projects-Deep-Learning\\lightvision\\outputs\\logs\\pruned_history.json'
qat_model_history_path = 'C:\\Users\\enapa\\Desktop\\Academic City\\Level 400\\DeepExam\\projects-Deep-Learning\\lightvision\\outputs\\logs\\qat_history.json'
resnet18_model_history_path = 'C:\\Users\\enapa\\Desktop\\Academic City\\Level 400\\DeepExam\\projects-Deep-Learning\\lightvision\\outputs\\logs\\resnet18_history.pkl'
teacher_model_history_path = 'C:\\Users\\enapa\\Desktop\\Academic City\\Level 400\\DeepExam\\projects-Deep-Learning\\lightvision\\outputs\\logs\\teacher_training_history.json'

OUTPUT_DIR = 'C:\\Users\\enapa\\Desktop\\Academic City\\Level 400\\DeepExam\\projects-Deep-Learning\\lightvision\\outputs\\plots'


In [18]:
#Calling the plotting function for each model
history_files = [
    (distilled_model_history_path, os.path.join(OUTPUT_DIR, 'distilled_model_history.png')),
    (pruned_model_history_path, os.path.join(OUTPUT_DIR, 'pruned_model_history.png')),
    (qat_model_history_path, os.path.join(OUTPUT_DIR, 'qat_model_history.png')),
    (resnet18_model_history_path, os.path.join(OUTPUT_DIR, 'resnet18_student_model_history.png')),
    (teacher_model_history_path, os.path.join(OUTPUT_DIR, 'teacher_model_history.png')),
]

for history_path, output_path in history_files:
    if os.path.exists(history_path):
        try:
            plot_training_history(history_path, output_path)
            print(f"✓ Plotted {os.path.basename(history_path)} → {os.path.basename(output_path)}")
        except Exception as e:
            print(f"✗ Error plotting {os.path.basename(history_path)}: {e}")
    else:
        print(f"⚠ Skipping {os.path.basename(history_path)} - file not found")

✓ Plotted distillation_history.json → distilled_model_history.png
✓ Plotted pruned_history.json → pruned_model_history.png
✓ Plotted qat_history.json → qat_model_history.png
✓ Plotted qat_history.json → qat_model_history.png
✓ Plotted resnet18_history.pkl → resnet18_student_model_history.png
✓ Plotted teacher_training_history.json → teacher_model_history.png
✓ Plotted resnet18_history.pkl → resnet18_student_model_history.png
✓ Plotted teacher_training_history.json → teacher_model_history.png
